In [46]:
import sys
import csv
import requests
import re
from bs4 import BeautifulSoup

In [50]:
def load_page(url):
    with requests.get(url) as f:
        page = f.text
    return page

In [61]:
def get_song_info(url):
    song_page = BeautifulSoup(load_page(url), 'lxml')                  # Parse the requested page
    interesting_html = song_page.find('div')        # Get the part of the page where the interesting information is 'stored'
    if not interesting_html:                                           # Check if an article tag was found, not all pages have one
        print('No information availible for song at {}'.format(url), file=sys.stderr)
        return {}           
# Return an empty dictionary
    content = interesting_html.find('ringtone')
    for row in content.find_all(class_ ='ringtone'):
        credits = get_element_text(interesting_html.find(class_='feat')) # title = interesting_html.find('h2').text.strip() # We already have this information
        lyrics = get_element_text(interesting_html.find(class_='div'))
        lyrics = lyrics.replace('\r\n', '\n')  
        lyrics = re.sub('\t\t+', '\n', lyrics)
    return {'credits': credits, 'lyrics': lyrics}     

In [62]:
song_url = 'https://www.azlyrics.com/lyrics/charlieputh/marvingaye.html'
song_info = get_song_info(song_url)
for key, value in song_info.items():
    if key == 'lyrics':
        print(value)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [63]:
def get_songs(url):
    index_page = BeautifulSoup(load_page(url), 'lxml')        # Parse the page
    items = index_page.find(id='listAlbum')                   # Get the list from the webpage
    if not items:                                             # If the webpage does not contain the list, we should exit
        print('Something went wrong!', file=sys.stderr)
        sys.exit()
    data = []
    for row in items.find_all(class_='listalbum-item'):          # Go over the entries line-by-line
        # Get the text in the first element with the class 'song'
        # Note: we use class_ with an underscore, as class is already a
        # reserved python keyword. This means we can't use it as a
        # variable name.
        song = row.find('a').text.strip() # Link text
        link = row.find('a').get('href')  # Link url
        #album_name = row.find(class_='release').text.strip()
        #first_played = row.find_all(class_='played')[0].text.strip()
        #last_played = row.find_all(class_='played')[1].text.strip()
       # times_played = row.find(class_='times').text.strip()
        data.append({                                         # Store the data in a dictionary, and add that to our list
                     'listalbum-item': song,
                     'listalbum-item': link,
                    # 'album name': album_name,
                    # 'first played': first_played,
                    # 'last played': last_played,
                    # 'times played': times_played
                    })
    return data

In [64]:
index_url = 'https://www.azlyrics.com/c/charlieputh.html'         # Contains a list of songs
song_data = get_songs(index_url)                      # Get songs with metadata
for row in song_data:
    print('Scraping info on {}.'.format(row['listalbum-item']) # Might be useful for debugging
    url = row['link']
    song_info = get_song_info(url)                    # Get lyrics and credits for this song, if available
    for key, value in song_info.items():
        row[key] = value        

SyntaxError: invalid syntax (1132005124.py, line 5)